In [1]:
import numpy as np
import pandas as pd
import matplotlib
import anndata as ad
import pickle

import plotly.io as pio
pio.renderers.default = 'iframe'

import plotly.express as px

In [11]:
#data_path = "./data/230811_Combined_TPM_clean_Tumor_filtered_ALL_T1_T2_INDIVIDUAL_PATIENTS.tsv"
#data_path = "./data/Combined_ALL.tsv"
data_path = "./data/Combined_ALL.tsv"
umap_path = "./data/Pancan_10f_UMAP_COORD.csv"

In [16]:
umap_df = pd.read_csv(umap_path, sep = ",")
umap_df = umap_df.set_index("Sample")
umap_df = umap_df.rename_axis("sample_name")
data_df = pd.read_csv(data_path, sep = "\t")

/var/folders/fn/yx26jgsj6d13mj5sdbhzxkr80000gn/T/ipykernel_11702/2499769177.py:4: DtypeWarning:

Columns (58056) have mixed types. Specify dtype option on import or set low_memory=False.



In [17]:
umap_df

,X_umap1,X_umap2
sample_name,,
IPIBLAD005.T1,-65.374725,-8.527859
IPIBLAD006.T1,2.978980,62.404606
IPIBLAD013.T1,5.030876,36.661865
IPIBLAD014.T1,-19.044546,46.680540
IPIBLAD019.T1,-36.074142,27.780592
...,...,...
IPISRC011.T1,-61.952957,-3.345270
IPISRC012.T1,-14.396193,35.802000
IPISRC013.T1,9.932845,64.804405


In [18]:
data_df

,sample_name,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,...,MIR3202.2,X.15278,X.15279,HSFX3,VINAC1P,MIR4481,Compartment,Patient,Indication,Archetype
1,IPIADR001.T1,0.66,0.0,10.42,8.00,5.97,16.38,4.36,8.51,56.51,...,0,0.0,1.58,0.00,0.0,0.0,T_cell,IPIADR001,ADR,NaN
2,IPIADR003.T1,1.15,0.0,9.89,3.97,9.80,31.72,7.52,14.40,52.40,...,0,0.0,0.62,0.13,0.0,0.0,T_cell,IPIADR003,ADR,NaN
3,IPIBLAD005.T1,0.10,0.0,13.54,3.81,20.18,3.55,1.92,7.27,115.02,...,0,0.0,3.36,0.00,0.0,0.0,T_cell,IPIBLAD005,BLAD,IR CD8 Mono
4,IPIBLAD006.T1,0.04,0.0,18.32,24.71,21.24,18.64,1.89,2.00,215.53,...,0,0.0,0.00,0.35,0.0,0.0,T_cell,IPIBLAD006,BLAD,IS CD8
5,IPIBLAD010.T1,0.00,0.0,9.83,12.88,3.51,10.09,3.77,0.54,13.90,...,0,0.0,3.52,0.00,0.0,0.0,T_cell,IPIBLAD010,BLAD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2682,IPISRC022.T1,0.09,0.0,17.79,9.49,9.82,0.70,0.48,23.40,24.38,...,0,0.0,3.67,0.09,0.0,0.0,Treg,IPISRC022,SRC,NaN
2692,IPISRC023.T1,0.05,0.0,18.94,5.05,16.73,2.33,1.71,16.60,27.53,...,0,0.0,0.56,4.33,0.0,0.0,Treg,IPISRC023,SRC,NaN
2702,IPISRC025.T1,0.00,0.0,11.55,5.39,17.33,0.15,0.27,32.61,39.74,...,0,0.0,0.00,0.00,0.0,0.0,Treg,IPISRC025,SRC,NaN
2715,IPISRC040.T1,2.89,0.0,10.51,7.71,4.59,7.00,2.76,9.23,33.82,...,0,0.0,0.00,0.00,0.0,0.0,Treg,IPISRC040,SRC,NaN


In [19]:
new_cols = ["sample_name", "Compartment", "Patient", "Indication", "Archetype"]
rest_of_cols = list(set(data_df.columns) - set(new_cols))
sorted_df = data_df[new_cols + rest_of_cols]
sorted_df

,sample_name,Compartment,Patient,Indication,Archetype,ENSG00000269998,X.225,GADD45B,SYCE1L,PIP4K2B,...,RPS25P7,TSSC2,CACNA1C.AS1,X.12251,GAPDHP47,SEC22C,CST9,Y_RNA.102,ACTRT1P1,X.4902
1,IPIADR001.T1,T_cell,IPIADR001,ADR,NaN,0.00,0.0,80.72,0.40,8.92,...,0.0,0.00,0.16,0.00,0.00,27.52,0.0,0.0,0.0,0.0
2,IPIADR003.T1,T_cell,IPIADR003,ADR,NaN,0.00,0.0,159.19,0.00,4.20,...,0.0,0.00,1.16,0.00,0.00,14.28,0.0,0.0,0.0,0.0
3,IPIBLAD005.T1,T_cell,IPIBLAD005,BLAD,IR CD8 Mono,0.00,0.0,162.95,0.00,21.93,...,0.0,0.12,0.00,0.00,1.85,20.58,0.0,0.0,0.0,0.0
4,IPIBLAD006.T1,T_cell,IPIBLAD006,BLAD,IS CD8,0.00,0.0,142.42,0.00,1.97,...,0.0,0.17,0.00,0.00,0.00,14.43,0.0,0.0,0.0,0.0
5,IPIBLAD010.T1,T_cell,IPIBLAD010,BLAD,NaN,0.00,0.0,116.69,0.00,3.92,...,0.0,0.28,0.00,30.48,0.00,11.21,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2682,IPISRC022.T1,Treg,IPISRC022,SRC,NaN,0.00,0.0,53.15,8.92,22.99,...,0.0,0.33,0.00,0.00,0.26,23.14,0.0,0.0,0.0,0.0
2692,IPISRC023.T1,Treg,IPISRC023,SRC,NaN,0.00,0.0,64.87,3.69,12.13,...,0.0,0.09,0.16,0.00,6.65,24.25,0.0,0.0,0.0,0.0
2702,IPISRC025.T1,Treg,IPISRC025,SRC,NaN,0.00,0.0,10.10,4.66,9.59,...,0.0,0.62,0.00,0.00,0.00,33.30,0.0,0.0,0.0,0.0
2715,IPISRC040.T1,Treg,IPISRC040,SRC,NaN,0.31,0.0,26.06,0.00,9.53,...,0.0,0.14,0.00,0.00,0.00,11.15,0.0,0.0,0.0,0.0


In [38]:
multiind = sorted_df.set_index(["Patient","sample_name","Indication","Compartment","Archetype"]).sort_index()
merged_df = pd.merge(multiind,umap_df,left_index=True,right_index=True,how="outer")

In [39]:
merged_df.to_pickle("master_df.pi")

In [40]:
merged_df

LYPD1  SNORA73B  \
Patient   sample_name  Indication Compartment Archetype                    
IPIADR001 IPIADR001.T1 ADR        Myeloid     NaN         0.00      0.00   
                                  Stroma      NaN         0.88      0.00   
                                  T_cell      NaN         0.00      2.16   
                                  Tumor       NaN         0.00      0.00   
IPIADR002 IPIADR002.T1 ADR        Stroma      NaN         1.32      0.00   
...                                                        ...       ...   
IPISRC040 IPISRC040.T1 SRC        Treg        NaN         3.07      0.00   
IPISRC042 IPISRC042.T1 SRC        Myeloid     NaN         0.09      0.00   
                                  Stroma      NaN         0.00      0.00   
                                  T_cell      NaN         0.06      0.00   
                                  Treg        NaN         0.10      0.00   

                                                         ARHGEF11  X.12550  \
Patient   sample_name  Indication Compartment Archetype                      
IPIADR001 IPIADR001.T1 ADR        Myeloid     NaN           36.52        0   
                                  Stroma      NaN           34.11        0   
                                  T_cell      NaN            2.78        0   
                                  Tumor       NaN           10.90        0   
IPIADR002 IPIADR002.T1 ADR        Stroma      NaN           45.86        0   
...                                                           ...      ...   
IPISRC040 IPISRC040.T1 SRC        Treg        NaN           10.30        0   
IPISRC042 IPISRC042.T1 SRC        Myeloid     NaN            5.96        0   
                                  Stroma      NaN           26.03        0   
                                  T_cell      NaN            1.31        0   
                                  Treg        NaN            0.03        0   

                                                         FCF1P4  X.6677  \
Patient   sample_name  Indication Compartment Archetype                   
IPIADR001 IPIADR001.T1 ADR        Myeloid     NaN           0.0    2.80   
                                  Stroma      NaN           0.0    0.54   
                                  T_cell      NaN           0.0    2.30   
                                  Tumor       NaN           0.0    0.00   
IPIADR002 IPIADR002.T1 ADR        Stroma      NaN           0.0    0.00   
...                                                         ...     ...   
IPISRC040 IPISRC040.T1 SRC        Treg        NaN           0.0    0.00   
IPISRC042 IPISRC042.T1 SRC        Myeloid     NaN           0.0    0.00   
                                  Stroma      NaN           0.0    0.00   
                                  T_cell      NaN           0.0    0.00   
                                  Treg        NaN           0.0    0.00   

                                                         X.4243  ANAPC7  \
Patient   sample_name  Indication Compartment Archetype                   
IPIADR001 IPIADR001.T1 ADR        Myeloid     NaN           0.0   16.41   
                                  Stroma      NaN           0.0   34.24   
                                  T_cell      NaN           0.0   22.38   
                                  Tumor       NaN           0.0   71.41   
IPIADR002 IPIADR002.T1 ADR        Stroma      NaN           0.0   51.16   
...                                                         ...     ...   
IPISRC040 IPISRC040.T1 SRC        Treg        NaN           0.0   19.06   
IPISRC042 IPISRC042.T1 SRC        Myeloid     NaN           0.0   16.94   
                                  Stroma      NaN           0.0   13.18   
                                  T_cell      NaN           0.0   27.47   
                                  Treg        NaN           0.0   10.39   

                                                          RGS4  Y_RNA.54  ...  \
Patient   sample_nam

In [3]:
master_df = pd.read_pickle("master_df.pi")
pd.options.plotting.backend = 'plotly'

In [16]:
tidy_df = master_df.reset_index()
tidy_df

,Patient,sample_name,Indication,Compartment,Archetype,ACSM1,LYVE1,TESPA1,PTPN18,NMNAT3,...,X.8677,X.12471,NPM1P12,LINC02003,LINC02995,PEX11G,X.2538,FGF18,X_umap1,X_umap2
0,IPIADR001,IPIADR001.T1,ADR,Myeloid,NaN,0.00,150.38,1.26,201.53,0.81,...,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.16,NaN,NaN
1,IPIADR001,IPIADR001.T1,ADR,Stroma,NaN,0.00,10.57,0.16,2.21,0.00,...,0.0,0.00,0.00,0.0,0.00,0.81,0.0,0.03,NaN,NaN
2,IPIADR001,IPIADR001.T1,ADR,T_cell,NaN,0.89,2.33,75.71,41.45,0.27,...,0.0,0.00,0.31,0.0,0.00,0.00,0.0,0.04,NaN,NaN
3,IPIADR001,IPIADR001.T1,ADR,Tumor,NaN,0.00,31.84,0.00,2.16,0.00,...,0.0,0.00,0.36,0.0,0.00,0.00,0.0,0.00,NaN,NaN
4,IPIADR002,IPIADR002.T1,ADR,Stroma,NaN,0.00,7.90,1.09,26.19,11.69,...,0.0,0.00,0.76,0.0,0.00,0.00,0.0,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,IPISRC040,IPISRC040.T1,SRC,Treg,NaN,0.00,2.49,30.94,26.03,0.14,...,0.0,0.27,0.00,0.0,0.27,0.00,0.0,0.00,NaN,NaN
1417,IPISRC042,IPISRC042.T1,SRC,Myeloid,NaN,0.14,1.66,4.45,67.78,0.00,...,0.0,0.00,0.00,0.0,0.00,0.13,0.0,0.04,NaN,NaN
1418,IPISRC042,IPISRC042.T1,SRC,Stroma,NaN,0.00,2.35,0.06,43.60,12.45,...,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,NaN,NaN
1419,IPISRC042,IPISRC042.T1,SRC,T_cell,NaN,0.00,0.10,47.82,48.80,1.23,...,0.0,0.00,0.44,0.0,0.39,0.00,0.0,0.06,NaN,NaN


In [27]:
tidy_df[tidy_df["sample_name"].str.contains(".N")]

,Patient,sample_name,Indication,Compartment,Archetype,ACSM1,LYVE1,TESPA1,PTPN18,NMNAT3,...,X.8677,X.12471,NPM1P12,LINC02003,LINC02995,PEX11G,X.2538,FGF18,X_umap1,X_umap2
291,IPIGYN003,IPIGYN003.T1,GYN,Myeloid,MC DC1,0.00,14.83,0.00,129.04,4.54,...,0.0,0.0,1.30,0.0,0.00,0.00,0.00,0.00,-8.942838,28.28356
292,IPIGYN003,IPIGYN003.T1,GYN,Stroma,MC DC1,0.00,0.22,0.00,62.04,0.00,...,0.0,0.0,1.03,0.0,0.00,0.00,0.00,1.51,-8.942838,28.28356
293,IPIGYN004,IPIGYN004.T1,GYN,Stroma,NaN,0.00,4.78,0.09,22.03,0.00,...,0.0,0.0,1.09,0.0,0.00,0.00,0.16,0.35,NaN,NaN
294,IPIGYN007,IPIGYN007.T1,GYN,Myeloid,NaN,0.00,8.69,3.28,130.09,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,NaN
295,IPIGYN007,IPIGYN007.T1,GYN,Treg,NaN,0.00,0.23,36.09,71.11,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,IPIPNET023,IPIPNET023.T2,PNET,Myeloid,NaN,0.00,8.54,1.98,271.10,0.18,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,NaN
1335,IPIPNET023,IPIPNET023.T2,PNET,Stroma,NaN,0.00,19.60,0.39,14.01,6.70,...,0.0,0.0,0.00,0.0,7.13,0.08,0.00,0.00,NaN,NaN
1336,IPIPNET023,IPIPNET023.T2,PNET,T_cell,NaN,0.75,0.00,83.55,69.55,0.30,...,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,NaN
1337,IPIPNET023,IPIPNET023.T2,PNET,Treg,NaN,0.00,0.00,37.08,84.19,0.00,...,0.0,0.0,0.14,0.0,0.00,0.00,0.00,0.08,NaN,NaN


In [29]:
px.box(tidy_df,x=["Indication","Compartment"], y="LYVE1")

In [10]:
pd.options.plotting.backend = 'plotly'
fig = master_df[["LYVE1"]].groupby("Indication").plot.box()
fig.show()

AttributeError: 'Series' object has no attribute 'show'

# Iso-Seq Stuff

In [30]:
iso_data = pd.read_csv("/Users/hwismer/Documents/A_01/bcM0001_seurat/bcM0001_collapsed.annotated.info.csv",
                      sep = "\t")

In [32]:
iso_data["category"]

0           novel_not_in_catalog
1              full-splice_match
2           novel_not_in_catalog
3           novel_not_in_catalog
4           novel_not_in_catalog
                    ...         
19101897       full-splice_match
19101898       full-splice_match
19101899       full-splice_match
19101900       full-splice_match
19101901    novel_not_in_catalog
Name: category, Length: 19101902, dtype: object

In [ ]:
iso_data[iso_data["gene"].str.contains("NCAM1")]

In [ ]:
iso_data[iso_data["gene"].str.contains("NCAM1")].groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')

In [ ]:
iso_data.groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')

In [ ]:
iso_data[iso_data["gene"].str.contains("IGF2")].groupby("category").size().plot(kind="pie", y = "category",autopct='%.2f')